In [1]:
'''
This experiment is designed to test the following hypothesis:
- An architecture that can adaptively switch between different sparsity optimizations during training is more energy efficient that architectures that only use one sparsity optimization
- In particular, the leakage energy cost of idle sparsity optimization components is less than the energy savings from switching to a more optimal sparsity optimization

We make the following assumptions:
- Base Eyeriss-v2 PE is used for all experiments, only sparse optimizations are modified
- Weight sparsities vary between 0% (random initialiation) and the final weight sparsity of the model
- Input sparsities vary between the input sparsities at random initialization and that sparsities produced by the model
- PE is directly connected to a sufficiently large DRAM to store all data

We will test on the following layers
- Foward pass and Backward pass (back propagation) on {some subset} of layers
'''

'\nThis experiment is designed to test the following hypothesis:\n- An architecture that can adaptively switch between different sparsity optimizations during training is more energy efficient that architectures that only use one sparsity optimization\n- In particular, the leakage energy cost of idle sparsity optimization components is less than the energy savings from switching to a more optimal sparsity optimization\n\nWe make the following assumptions:\n- Base Eyeriss-v2 PE is used for all experiments, only sparse optimizations are modified\n- Weight sparsities vary between 0% (random initialiation) and the final weight sparsity of the model\n- Input sparsities vary between the input sparsities at random initialization and that sparsities produced by the model\n- PE is directly connected to a sufficiently large DRAM to store all data\n\nWe will test on the following layers\n- Foward pass and Backward pass (back propagation) on {some subset} of layers\n'

In [2]:
import yaml, inspect, os, sys, subprocess, pprint, shutil, argparse, copy
from utils import *

In [3]:
base_dir = os.path.join(os.getcwd(),"output-sparsity-experiments")

In [4]:
'''
Each experiment contains
- design 
    - one folder for each {sparsity_opt}
        - arch-{sparsity_opt}.yaml : architecture description, where sparsity_opt is one of dense, gating-only, skipping-only
        - sparsity-{sparsity_opt}.yaml : sparse optimizations description
        
    - compound_components.yaml : component description, including gating and skipping
    
    
- ert-art : for files produced by Accelergy, later modified to account for gating / skipping overhead
    - one folder for each of dense, gating-only, skipping-only
        - ART-{sparsity_opt}.yaml 
        - ART-{sparsity_opt}_summary.yaml
        - ERT-{sparsity_opt}.yaml
        - ERT-{sparsity_opt}_summary.yaml
    
- mapper 
    - constraints:
        - one constraints file for each layer, file format {layer}-constraints.yaml
    - mapper.yaml : mapper optimization controls
    
- mapping 
    - one folder for each sparsity_opt
        - one folder for each layer, folder name {layer}
            - one mapping for each sparsity level, file format {sparsity_opt}-{layer}-{input_sparsity}-{weight_sparsity}.map.yaml
    
- raw-stats
    - one folder for each sparsity_opt
        - one folder for each layer, folder name {layer}
            - one stats file for each sparsity level, file format {sparsity_opt}-{layer}-{input_sparsity}-{weight_sparsity}.stats.txt 
- workloads
    - one folder for each layer, folder name {layer}
        - one workload for each sparsity level in each layer, file format {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml

- raw-workloads
    - one workload for each layer, file format {layer}-raw-workload.yaml
    
- sparsities.csv : columns of (layer,min_input_sparsity,max_input_sparsity, min_weight_sparsity, max_weight_sparsity), one row per layer
'''


'\nEach experiment contains\n- design \n    - one folder for each {sparsity_opt}\n        - arch-{sparsity_opt}.yaml : architecture description, where sparsity_opt is one of dense, gating-only, skipping-only\n        - sparsity-{sparsity_opt}.yaml : sparse optimizations description\n        \n    - compound_components.yaml : component description, including gating and skipping\n    \n    \n- ert-art : for files produced by Accelergy, later modified to account for gating / skipping overhead\n    - one folder for each of dense, gating-only, skipping-only\n        - ART-{sparsity_opt}.yaml \n        - ART-{sparsity_opt}_summary.yaml\n        - ERT-{sparsity_opt}.yaml\n        - ERT-{sparsity_opt}_summary.yaml\n    \n- mapper \n    - constraints:\n        - one constraints file for each layer, file format {layer}-constraints.yaml\n    - mapper.yaml : mapper optimization controls\n    \n- mapping \n    - one folder for each sparsity_opt\n        - one folder for each layer, folder name {lay

In [5]:
import os

def ensure_path_exists(filepath):
    """Ensure that all intermediate directories and the file itself exist."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    if not os.path.exists(filepath):
        # Create an empty file
        open(filepath, 'a').close()

# === DESIGN ===

def get_arch_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "design", f"arch-{sparsity_opt}.yaml")
    ensure_path_exists(path)
    return path

def get_compound_components_yaml(base_dir):
    path = os.path.join(base_dir, "design", "compound_components.yaml")
    ensure_path_exists(path)
    return path

def get_sparsity_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "design", f"sparsity-{sparsity_opt}.yaml")
    ensure_path_exists(path)
    return path

# === ERT-ART ===

def get_art_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "ert-art", sparsity_opt, f"ART-{sparsity_opt}.yaml")
    ensure_path_exists(path)
    return path

def get_art_summary_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "ert-art", sparsity_opt, f"ART-{sparsity_opt}_summary.yaml")
    ensure_path_exists(path)
    return path

def get_ert_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "ert-art", sparsity_opt, f"ERT-{sparsity_opt}.yaml")
    ensure_path_exists(path)
    return path

def get_ert_summary_yaml(base_dir, sparsity_opt):
    path = os.path.join(base_dir, "ert-art", sparsity_opt, f"ERT-{sparsity_opt}_summary.yaml")
    ensure_path_exists(path)
    return path

# === MAPPER ===

def get_mapper_yaml(base_dir):
    path = os.path.join(base_dir, "mapper", "mapper.yaml")
    ensure_path_exists(path)
    return path

def get_constraints_yaml(base_dir, layer):
    """
    Get the path to the constraints file for a layer.
    If it doesn't exist, copy from default-constraints.yaml.
    """
    mapper_dir = os.path.join(base_dir, "mapper")
    constraints_dir = os.path.join(base_dir, "mapper", "constraints")
    filepath = os.path.join(constraints_dir, f"{layer}-constraints.yaml")
    default_forward_filepath = os.path.join(mapper_dir, "default-constraints-forward.yaml")
    default_backward_filepath = os.path.join(mapper_dir, "default-constraints-backward.yaml")
    
    default_filepath = (default_backward_filepath if ("backward" in layer) else default_forward_filepath)

    os.makedirs(constraints_dir, exist_ok=True)

    if not os.path.exists(filepath):
        if os.path.exists(default_filepath):
            shutil.copy(default_filepath, filepath)
        else:
            # If default doesn't exist, just create an empty file
            open(filepath, 'a').close()
    
    return filepath

# === MAPPING ===

def get_mapping_yaml(base_dir, sparsity_opt, layer, input_sparsity, weight_sparsity):
    path = os.path.join(base_dir, "mapping", sparsity_opt, layer,
                        f"{sparsity_opt}-{layer}-{input_sparsity}-{weight_sparsity}.map.yaml")
    ensure_path_exists(path)
    return path

# === RAW STATS ===

def get_stats_file(base_dir, sparsity_opt, layer, input_sparsity, weight_sparsity):
    path = os.path.join(base_dir, "raw-stats", sparsity_opt, layer,
                        f"{sparsity_opt}-{layer}-{input_sparsity}-{weight_sparsity}.stats.txt")
    ensure_path_exists(path)
    return path

# === WORKLOADS ===

def get_workload_yaml(base_dir, layer, input_sparsity, weight_sparsity):
    path = os.path.join(base_dir, "workloads", layer,
                        f"{layer}-{input_sparsity}-{weight_sparsity}-workload.yaml")
    ensure_path_exists(path)
    return path

# === RAW WORKLOADS ===

def get_raw_workload_yaml(base_dir, layer):
    path = os.path.join(base_dir, "raw-workloads", f"{layer}-raw-workload.yaml")
    ensure_path_exists(path)
    return path

# === SPARSITY CSV ===

def get_sparsities_csv(base_dir):
    path = os.path.join(base_dir, "sparsities.csv")
    ensure_path_exists(path)
    return path


# === LAYER INFORMATION JSON ===

def get_layer_information_csv(base_dir):
    path = os.path.join(base_dir, "layer_information.json")
    ensure_path_exists(path)
    return path



In [6]:
import json
import csv
from typing import List, Dict, Any

def load_json(filename: str) -> Dict[str, Any]:
    with open(filename, 'r') as f:
        return json.load(f)

def extract_layer_info(layer_data: Dict[str, Any], backward = False) -> Dict[str, Any]:
    misc = layer_data.get("miscellaneous", {})
    
    return {
        "layer": misc.get("name", "")+"-backward"*(1 if backward else 0),
        "input_density_5": layer_data.get("input_density_5", "1.0"),
        "input_density_50": layer_data.get("input_density_50", "1.0"),
        "input_density_95": layer_data.get("input_density_95", "1.0"),
        "weight_density": layer_data.get("weight_density", "1.0"),
        "output_density_5": layer_data.get("output_density_5", "1.0"),
        "output_density_50": layer_data.get("output_density_50", "1.0"),
        "output_density_95": layer_data.get("output_density_95", "1.0"),
        "input_shape_batch_N": layer_data.get("input_shape", [None]*4)[0],
        "input_shape_rows_H": layer_data.get("input_shape", [None]*4)[1],
        "input_shape_cols_W": layer_data.get("input_shape", [None]*4)[2],
        "input_shape_channels_C": layer_data.get("input_shape", [None]*4)[3],
        "output_shape_batch_N": layer_data.get("output_shape", [None]*4)[0],
        "output_shape_rows_F": layer_data.get("output_shape", [None]*4)[1],
        "output_shape_cols_E": layer_data.get("output_shape", [None]*4)[2],
        "output_shape_channels_M": layer_data.get("output_shape", [None]*4)[3],
        "filters": misc.get("filters", ""),
        "depth_multiplier": misc.get("depth_multiplier", ""),
        "kernel_size_rows_S": misc.get("kernel_size", [None, None])[0],
        "kernel_size_cols_R": misc.get("kernel_size", [None, None])[1],
        "strides_rows_Hstride": misc.get("strides", [None, None])[0],
        "strides_cols_Wstride": misc.get("strides", [None, None])[1],
        "dilation_rate_rows_Hdilation": misc.get("dilation_rate", [None, None])[0],
        "dilation_rate_cols_Wdilation": misc.get("dilation_rate", [None, None])[1],
    }


def parse_all_layers(json_data: Dict[str, Any]) -> List[Dict[str, Any]]:
    return [extract_layer_info(layer) for key, layer in sorted(json_data.items(), key=lambda x: int(x[0]))] + [extract_layer_info(layer,backward = True) for key, layer in sorted(json_data.items(), key=lambda x: int(x[0]))] 

def write_to_csv(data: List[Dict[str, Any]], output_filename: str) -> None:
    if not data:
        return
    
    fieldnames = list(data[0].keys())
    
    with open(output_filename, mode='w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

def process_json_to_csv(json_filename: str, csv_filename: str) -> None:
    json_data = load_json(json_filename)
    parsed_data = parse_all_layers(json_data)
    write_to_csv(parsed_data, csv_filename)

# Example usage:
# process_json_to_csv('information (1).json', 'mobilenet_layers.csv')
process_json_to_csv(
    json_filename = get_layer_information_csv(base_dir),
    csv_filename = get_sparsities_csv(base_dir)
)

In [7]:
import csv

def parse_sparsities_csv(filename):
    """
    Parses a sparsities.csv file and returns a list of dictionaries, one for each row.
    
    Each dictionary has keys:
        - 'layer'
        - 'min_input_sparsity'
        - 'max_input_sparsity'
        - 'min_weight_sparsity'
        - 'max_weight_sparsity'
    """
    sparsity_data = []
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sparsity_data.append({
                'layer': row['layer'],
                'min_input_sparsity': float(row['input_density_5']),
                'max_input_sparsity': float(row['input_density_95']),
                'min_weight_sparsity': float(row['weight_density']),
                'max_weight_sparsity': float(row['weight_density']),
                'min_output_sparsity': float(row['output_density_5']),
                'max_output_sparsity': float(row['output_density_95']),
            })
    return sparsity_data


In [8]:
import pandas as pd
import yaml
from typing import Dict, Any

def load_csv(csv_filename: str) -> pd.DataFrame:
    return pd.read_csv(csv_filename)

def create_forward_pass_yaml_dict(layer: Dict[str, Any]) -> Dict[str, Any]:
    yaml_structure = {
        'problem': {
            'shape': {
                'name': "grouped-CONV",
                'dimensions': ['R', 'S', 'E', 'F', 'C', 'M', 'N', 'G'],
                'coefficients': [
                    {'name': 'Wstride', 'default': 1},
                    {'name': 'Hstride', 'default': 1},
                    {'name': 'Wdilation', 'default': 1},
                    {'name': 'Hdilation', 'default': 1},
                ],
                'data-spaces': [
                    {'name': 'Weights', 'projection': [[['C']], [['M']], [['G']], [['R']], [['S']]]},
                    {'name': 'Inputs', 'projection': [[['N']], [['C']], [['G']], [['R', 'Wdilation'], ['E', 'Wstride']], [['S', 'Hdilation'], ['F', 'Hstride']]]},
                    {'name': 'Outputs', 'projection': [[['N']], [['G']], [['M']], [['F']], [['E']]], 'read-write': True}
                ]
            },
            'instance': {
                'M': int(layer.get('filters_M', layer['input_shape_channels_C'])),
                'E': int(layer['output_shape_cols_E']),
                'F': int(layer['output_shape_rows_F']),
                'R': int(layer["kernel_size_cols_R"]),
                'S': int(layer["kernel_size_rows_S"]),
                'N': int(layer['input_shape_batch_N']),
                'G': 1,
                'C': int(layer['input_shape_channels_C']),
                'densities': {
                    'Inputs': {
                        'distribution': 'hypergeometric',
                        'density': float(layer['input_density_50'])
                    },
                    'Weights': {
                        'distribution': 'hypergeometric',
                        'density': float(layer.get('weight_density', 1.0))
                    }
                }
            }
        }
    }
    
    # Overwrite Wstride and Hstride if needed
    for coef in yaml_structure['problem']['shape']['coefficients']:
        if coef['name'] == 'Wstride':
            coef['default'] = int(layer['strides_cols_Wstride'])
        if coef['name'] == 'Hstride':
            coef['default'] = int(layer['strides_rows_Hstride'])

    return yaml_structure

def create_backward_pass_yaml_dict(layer: Dict[str, Any]) -> Dict[str, Any]:
    yaml_structure = {
        'problem': {
            'shape': {
                'name': "grouped-CONV-backpropagation",
                'dimensions': ['R', 'S', 'E', 'F', 'C', 'M', 'N', 'G'],
                'coefficients': [
                    {'name': 'Wstride', 'default': 1},
                    {'name': 'Hstride', 'default': 1},
                    {'name': 'Wdilation', 'default': 1},
                    {'name': 'Hdilation', 'default': 1},
                ],
                'data-spaces': [
                    # Outputs are gradients of weights
                    {'name': 'Outputs', 'projection': [[['C']], [['M']], [['G']], [['R']], [['S']]],  'read-write': True}, 
                    # Weights are the inputs
                    {'name': 'Weights', 'projection': [[['N']], [['C']], [['G']], [['R', 'Wdilation'], ['E', 'Wstride']], [['S', 'Hdilation'], ['F', 'Hstride']]]}, 
                    # Inputs are gradients of outputs
                    {'name': 'Inputs', 'projection': [[['N']], [['G']], [['M']], [['F']], [['E']]]} 
                ]
            },
            'instance': {
                'M': int(layer.get('filters_M', layer['input_shape_channels_C'])),
                'E': int(layer['output_shape_cols_E']),
                'F': int(layer['output_shape_rows_F']),
                'R': int(layer["kernel_size_cols_R"]),
                'S': int(layer["kernel_size_rows_S"]),
                'N': int(layer['input_shape_batch_N']),
                'G': 1,
                'C': int(layer['input_shape_channels_C']),
                'densities': {
                    'Inputs': { # Inputs are gradients of outputs
                        'distribution': 'hypergeometric',
                        'density': float(layer['output_density_50'])
                    },
                    'Weights': { # Weights are the inputs
                        'distribution': 'hypergeometric',
                        'density': float(layer['input_density_50'])
                    }
                }
            }
        }
    }
    
    # Overwrite Wstride and Hstride if needed
    for coef in yaml_structure['problem']['shape']['coefficients']:
        if coef['name'] == 'Wstride':
            coef['default'] = int(layer['strides_cols_Wstride'])
        if coef['name'] == 'Hstride':
            coef['default'] = int(layer['strides_rows_Hstride'])

    return yaml_structure

def save_yaml(yaml_data: Dict[str, Any], output_filename: str) -> None:
    with open(output_filename, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False)

def process_csv_to_yaml(base_dir : str, csv_filename: str) -> None:
    df = load_csv(csv_filename)
    for idx, row in df.iterrows():
        layer_dict = row.to_dict()
        print(row.to_dict()["layer"])
        if "backward" in row.to_dict()["layer"]:
            backward_yaml_data = create_backward_pass_yaml_dict(layer_dict)
            save_yaml(backward_yaml_data, get_raw_workload_yaml(base_dir , layer_dict["layer"]))
        else:
            forward_yaml_data = create_forward_pass_yaml_dict(layer_dict)
            save_yaml(forward_yaml_data, get_raw_workload_yaml(base_dir , layer_dict["layer"]))
        


process_csv_to_yaml(
    base_dir = base_dir,
    csv_filename = get_sparsities_csv(base_dir)
)


conv1
conv_dw_1
conv_pw_1
conv_dw_2
conv_pw_2
conv_dw_3
conv_pw_3
conv_dw_4
conv_pw_4
conv_dw_5
conv_pw_5
conv_dw_6
conv_pw_6
conv_dw_7
conv_pw_7
conv_dw_8
conv_pw_8
conv_dw_9
conv_pw_9
conv_dw_10
conv_pw_10
conv_dw_11
conv_pw_11
conv_dw_12
conv_pw_12
conv_dw_13
conv_pw_13
conv1-backward
conv_dw_1-backward
conv_pw_1-backward
conv_dw_2-backward
conv_pw_2-backward
conv_dw_3-backward
conv_pw_3-backward
conv_dw_4-backward
conv_pw_4-backward
conv_dw_5-backward
conv_pw_5-backward
conv_dw_6-backward
conv_pw_6-backward
conv_dw_7-backward
conv_pw_7-backward
conv_dw_8-backward
conv_pw_8-backward
conv_dw_9-backward
conv_pw_9-backward
conv_dw_10-backward
conv_pw_10-backward
conv_dw_11-backward
conv_pw_11-backward
conv_dw_12-backward
conv_pw_12-backward
conv_dw_13-backward
conv_pw_13-backward


In [9]:
import numpy as np

def generate_sparsity_grid(layer, min_input, max_input, min_weight, max_weight, input_spread, weight_spread):
    """
    Generate a list of dictionaries with sampled combinations of input and weight sparsity values.
    
    Args:
        layer (str): The name of the layer.
        min_input (float): Minimum input sparsity.
        max_input (float): Maximum input sparsity.
        min_weight (float): Minimum weight sparsity.
        max_weight (float): Maximum weight sparsity.
        input_spread (int): Number of evenly spaced values to sample for input sparsity.
        weight_spread (int): Number of evenly spaced values to sample for weight sparsity.
    
    Returns:
        List[Dict[str, float]]: List of dictionaries with fields:
            - 'layer'
            - 'input_sparsity'
            - 'weight_sparsity'
    """
    input_values = np.linspace(min_input, max_input, input_spread)
    weight_values = np.linspace(min_weight, max_weight, weight_spread)

    combinations = []
    for input_sparsity in input_values:
        for weight_sparsity in weight_values:
            combinations.append({
                "layer": layer,
                "input_sparsity": round(float(input_sparsity), 5),
                "weight_sparsity": round(float(weight_sparsity), 5),
            })

    return combinations


In [10]:
def modify_distributions(layer_yaml, input_sparsity = None, weight_sparsity = None, output_sparsity = None):
    new_layer_yaml = copy.deepcopy(layer_yaml)
    if input_sparsity is not None:
        new_layer_yaml['problem']['instance']['densities']['Inputs']['density'] = input_sparsity
    if weight_sparsity is not None:
        new_layer_yaml['problem']['instance']['densities']['Weights']['density'] = weight_sparsity
    if output_sparsity is not None:
        new_layer_yaml['problem']['instance']['densities']['Outputs']['density'] = weight_sparsity
    return new_layer_yaml

def generate_modified_sparsity(base_dir, combination):
    raw_workload_file = get_raw_workload_yaml(
        base_dir = base_dir,
        layer = combination["layer"]
    )
    raw_workload_yaml = get_yaml_object(
        yaml_filename = raw_workload_file
    )
    new_workload_yaml = modify_distributions(
        layer_yaml = raw_workload_yaml,
        input_sparsity = combination["input_sparsity"],
        weight_sparsity = combination["weight_sparsity"]
    )
    new_workload_file = get_workload_yaml(
        base_dir = base_dir,
        layer = combination["layer"],
        input_sparsity = combination["input_sparsity"], 
        weight_sparsity = combination["weight_sparsity"]
    )
    write_yaml_to_file(
        yaml_filename = new_workload_file,
        yaml_object = new_workload_yaml
    )
    
def generate_all_sparsity_combinations(sparsity_data, input_spread, weight_spread):
    all_combinations = []
    for sparsity_row in sparsity_data:
        if "backward" in sparsity_row["layer"]:
            combinations = generate_sparsity_grid(
                layer = sparsity_row["layer"], 
                min_input = sparsity_row["min_input_sparsity"], 
                max_input = sparsity_row["max_input_sparsity"], 
                min_weight = sparsity_row["min_weight_sparsity"], 
                max_weight = sparsity_row["max_weight_sparsity"], 
                input_spread = input_spread, 
                weight_spread = weight_spread
            )
        else:
            combinations = generate_sparsity_grid(
                layer = sparsity_row["layer"], 
                min_input = sparsity_row["min_output_sparsity"], 
                max_input = sparsity_row["max_output_sparsity"], 
                min_weight = sparsity_row["min_input_sparsity"], 
                max_weight = sparsity_row["max_input_sparsity"], 
                input_spread = input_spread, 
                weight_spread = weight_spread
            )
        all_combinations.extend(combinations)
    return all_combinations

def get_sparsity_data(base_dir):
    sparsity_file = get_sparsities_csv(base_dir);
    sparsity_data = parse_sparsities_csv(sparsity_file);
    return sparsity_data
        
def generate_all_modified_sparsities(base_dir, input_spread, weight_spread):
    sparsity_data = get_sparsity_data(base_dir)
    all_combinations = generate_all_sparsity_combinations(sparsity_data,input_spread, weight_spread)
    
    for combination in all_combinations:
        generate_modified_sparsity(base_dir,combination)
    

In [11]:


def generate_accelergy(base_dir, sparsity_opts):
    for sparsity_opt in sparsity_opts:
        arch_opt_file = get_arch_yaml(
            base_dir = base_dir, 
            sparsity_opt = sparsity_opt
        )
        component_file = get_compound_components_yaml(
            base_dir = base_dir
        )
        run_accelergy(
            arch=arch_opt_file,
            components=component_file
        )
        shutil.move(
            "ART.yaml", 
            get_art_yaml(base_dir,sparsity_opt)
        )
        shutil.move(
            "ART_summary.yaml", 
            get_art_summary_yaml(base_dir,sparsity_opt)
        )
        shutil.move(
            "ERT.yaml", 
            get_ert_yaml(base_dir,sparsity_opt)
        )
        shutil.move(
            "ERT_summary.yaml", 
            get_ert_summary_yaml(base_dir,sparsity_opt)
        )
        

In [12]:
def run_mappers(base_dir, sparsity_opts, all_combinations):
    for combination in all_combinations:
        for sparsity_opt in sparsity_opts:
            layer = combination["layer"]
            input_sparsity = combination["input_sparsity"]
            weight_sparsity = combination["weight_sparsity"]
            print(f"Modelling {sparsity_opt} for layer {layer} with iact sparsity = {input_sparsity} and weight sparsity = {weight_sparsity}")
            
            run_timeloop_mapper(
                arch    = get_arch_yaml(
                    base_dir = base_dir, 
                    sparsity_opt = sparsity_opt
                ),
                problem = get_workload_yaml(
                    base_dir = base_dir, 
                    layer = layer, 
                    input_sparsity = input_sparsity, 
                    weight_sparsity = weight_sparsity
                ), 
                mapper  = get_mapper_yaml(base_dir),
                pe_art  = get_art_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                pe_ert  = get_ert_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                config  = get_sparsity_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                constraints = get_constraints_yaml(
                    base_dir = base_dir,
                    layer = layer
                )
            )
            shutil.move(
                "timeloop-mapper.map.yaml",
                get_mapping_yaml(
                    base_dir = base_dir, 
                    sparsity_opt = sparsity_opt, 
                    layer = layer, 
                    input_sparsity = input_sparsity, 
                    weight_sparsity = weight_sparsity
                )
            )
            print(f"Completed {sparsity_opt} for layer {layer} with iact sparsity = {input_sparsity} and weight sparsity = {weight_sparsity}")
            


In [13]:
def run_models(base_dir, sparsity_opts, all_combinations):
    for combination in all_combinations:
        for sparsity_opt in sparsity_opts:
            layer = combination["layer"]
            input_sparsity = combination["input_sparsity"]
            weight_sparsity = combination["weight_sparsity"]
            print(f"Mapping {sparsity_opt} for layer {layer} with iact sparsity = {input_sparsity} and weight sparsity = {weight_sparsity}")
            
            run_timeloop_single(
                workload       =   get_workload_yaml(
                    base_dir = base_dir, 
                    layer = layer, 
                    input_sparsity = input_sparsity, 
                    weight_sparsity = weight_sparsity
                ),
                mapping        =   get_mapping_yaml(
                    base_dir, 
                    sparsity_opt, 
                    layer, 
                    input_sparsity, 
                    weight_sparsity
                ),
                config          =   get_sparsity_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                pe_art  = get_art_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                pe_ert  = get_ert_yaml(
                    base_dir = base_dir,
                    sparsity_opt = sparsity_opt 
                ),
                arch    = get_arch_yaml(
                    base_dir = base_dir, 
                    sparsity_opt = sparsity_opt
                ),
            )
            shutil.move(
                "timeloop-model.stats.txt",
                get_stats_file(
                    base_dir, 
                    sparsity_opt, 
                    layer, 
                    input_sparsity, 
                    weight_sparsity
                )
            )
            print(f"Completed {sparsity_opt} for layer {layer} with iact sparsity = {input_sparsity} and weight sparsity = {weight_sparsity}")
            


In [14]:
'''
(1) Generate raw-workloads for each layer of the neural network
(2) Generate sparsities.csv
(3) Using raw-workloads and sparsities.csv, generated workloads for each layer and sparsity


(4) For each {sparsity_opt}
    - Use accelergy on the following files to generate ART-{sparsity_opt}.yaml and ERT-{sparsity_opt}.yaml
        - arch-{sparsity_opt}.yaml
        - compound_components.yaml
        
(5) For each {sparsity_opt}, {layer}, {sparsity}
    - Use the timeloop-mapper on the following files to generate {sparsity_opt}-{layer}-{sparsity}.map.yaml:
        - ert-{sparsity_opt}, 
        - art-{sparsity_opt}.yaml, 
        - arch-{sparsity_opt}.yaml,
        - {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml
        - mapper.yaml
        - compound_components.yaml
        - {layer}-constraints.yaml
        
    - Run timeloop-model on the following files to generate {sparsity_opt}-{layer}-{sparsity}.stats.txt
        - ert-{sparsity_opt}, 
        - art-{sparsity_opt}.yaml, 
        - arch-{sparsity_opt}.yaml,
        - {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml
        - compound_components.yaml
        - {layer}-constraints.yaml


(6) For each {layer}, 
    - Compute a 3D graph of 
        - (Total Energy)/(Energy / Cycle) on the z (vertical) axis against 
        - Input and weight sparsity on the x and y axes 
        - One plane for each sparsity opt as well as the minimum of all 3
        
    - ???
'''

'\n(1) Generate raw-workloads for each layer of the neural network\n(2) Generate sparsities.csv\n(3) Using raw-workloads and sparsities.csv, generated workloads for each layer and sparsity\n\n\n(4) For each {sparsity_opt}\n    - Use accelergy on the following files to generate ART-{sparsity_opt}.yaml and ERT-{sparsity_opt}.yaml\n        - arch-{sparsity_opt}.yaml\n        - compound_components.yaml\n        \n(5) For each {sparsity_opt}, {layer}, {sparsity}\n    - Use the timeloop-mapper on the following files to generate {sparsity_opt}-{layer}-{sparsity}.map.yaml:\n        - ert-{sparsity_opt}, \n        - art-{sparsity_opt}.yaml, \n        - arch-{sparsity_opt}.yaml,\n        - {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml\n        - mapper.yaml\n        - compound_components.yaml\n        - {layer}-constraints.yaml\n        \n    - Run timeloop-model on the following files to generate {sparsity_opt}-{layer}-{sparsity}.stats.txt\n        - ert-{sparsity_opt}, \n        - a

In [15]:
input_spread = 1
weight_spread = 1
sparsity_opts = [
    "dense",
    "gating",
    "skipping",
]

In [16]:
'''
(3) Using raw-workloads and sparsities.csv, generated workloads for each layer and sparsity
'''
generate_all_modified_sparsities(
    base_dir = base_dir, 
    input_spread = input_spread, 
    weight_spread = weight_spread
)

In [17]:
'''
(4) For each {sparsity_opt}
    - Use accelergy on the following files to generate ART-{sparsity_opt}.yaml and ERT-{sparsity_opt}.yaml
        - arch-{sparsity_opt}.yaml
        - compound_components.yaml
'''
generate_accelergy(
    base_dir = base_dir, 
    sparsity_opts = sparsity_opts
)

    _                _                      
   / \   ___ ___ ___| | ___ _ __ __ _ _   _ 
  / _ \ / __/ __/ _ \ |/ _ \ '__/ _` | | | |
 / ___ \ (_| (_|  __/ |  __/ | | (_| | |_| |
/_/   \_\___\___\___|_|\___|_|  \__, |\__, |
                                |___/ |___/ 

Info: generating outputs according to the following specified output flags... 
 Please use the -f flag to update the preference (default to all output files) 
{'ERT': 1, 'ERT_summary': 1, 'ART': 1, 'ART_summary': 1, 'energy_estimation': 1, 'flattened_arch': 1}
Info: config file located: /home/workspace/.config/accelergy/accelergy_config.yaml 
config file content: 
 {'version': 0.3, 'compound_components': [], 'estimator_plug_ins': ['/usr/local/share/accelergy/estimation_plug_ins'], 'primitive_components': ['/usr/local/share/accelergy/primitive_component_libs'], 'table_plug_ins': {'roots': ['/usr/local/share/accelergy/estimation_plug_ins/accelergy-table-based-plug-ins/set_of_table_templates']}}
Info: Parsing file /home/wo

    _                _                      
   / \   ___ ___ ___| | ___ _ __ __ _ _   _ 
  / _ \ / __/ __/ _ \ |/ _ \ '__/ _` | | | |
 / ___ \ (_| (_|  __/ |  __/ | | (_| | |_| |
/_/   \_\___\___\___|_|\___|_|  \__, |\__, |
                                |___/ |___/ 

Info: generating outputs according to the following specified output flags... 
 Please use the -f flag to update the preference (default to all output files) 
{'ERT': 1, 'ERT_summary': 1, 'ART': 1, 'ART_summary': 1, 'energy_estimation': 1, 'flattened_arch': 1}
Info: config file located: /home/workspace/.config/accelergy/accelergy_config.yaml 
config file content: 
 {'version': 0.3, 'compound_components': [], 'estimator_plug_ins': ['/usr/local/share/accelergy/estimation_plug_ins'], 'primitive_components': ['/usr/local/share/accelergy/primitive_component_libs'], 'table_plug_ins': {'roots': ['/usr/local/share/accelergy/estimation_plug_ins/accelergy-table-based-plug-ins/set_of_table_templates']}}
Info: Parsing file /home/wo

In [ ]:
'''
(5) For each {sparsity_opt}, {layer}, {sparsity}
    - Use the timeloop-mapper on the following files to generate {sparsity_opt}-{layer}-{sparsity}.map.yaml:
        - ert-{sparsity_opt}, 
        - art-{sparsity_opt}.yaml, 
        - arch-{sparsity_opt}.yaml,
        - {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml
        - mapper.yaml
        - compound_components.yaml
        - {layer}-constraints.yaml
        
    - Run timeloop-model on the following files to generate {sparsity_opt}-{layer}-{sparsity}.stats.txt
        - ert-{sparsity_opt}, 
        - art-{sparsity_opt}.yaml, 
        - arch-{sparsity_opt}.yaml,
        - {layer}-{input_sparsity}-{weight_sparsity}-workload.yaml
        - compound_components.yaml
        - {layer}-constraints.yaml
'''

run_mapper_flag = True
run_model_flag = True

if run_mapper_flag:
    run_mappers(
        base_dir = base_dir, 
        sparsity_opts = sparsity_opts, 
        all_combinations = generate_all_sparsity_combinations(
            sparsity_data = get_sparsity_data(base_dir), 
            input_spread = input_spread, 
            weight_spread = weight_spread
        )
    )

if run_model_flag:
    run_models(
        base_dir = base_dir, 
        sparsity_opts = sparsity_opts, 
        all_combinations = generate_all_sparsity_combinations(
            sparsity_data = get_sparsity_data(base_dir), 
            input_spread = input_spread, 
            weight_spread = weight_spread
        )
    )

Modelling dense for layer conv1 with iact sparsity = 1.0 and weight sparsity = 1.0
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
Found Accelergy ERT (energy reference table), replacing internal energy model.
Found Accelergy ART (area reference table), replacing internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using all available hardware threads = 16
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension R = 4
  Factorization options along problem dimension S = 1
  Factorization options along problem dimension E = 1
  Factorization options along problem dimension F = 1
  Factorization options along problem dimension C = 4

[  1] Utilization = 1.00 | pJ/Algorithmic-Compute =  563.421 | pJ/Compute =  563.421 | L4[WIO] M3 F112 E112 S3 - L3[I] G1 - L2[W] R3 - L1[O] C3 - L0[I] G1 
[  2] Utilization = 1.00 | pJ/Algorithmic-Compute =  563.643 | pJ/Compute =  563.643 | L4[WIO] M3 F112 E112 S3 - L3[I] R3 - L2[W] G1 - L1[O] C3 - L0[I] G1 
[  6] Utilization = 1.00 | pJ/Algorithmic-Compute =  389.733 | pJ/Compute =  389.733 | L4[WIO] F112 E112 S3 - L3[I] R3 - L2[W] C3 - L1[O] M3 - L0[I] G1 
[ 13] Utilization = 1.00 | pJ/Algorithmic-Compute =  427.830 | pJ/Compute =  427.830 | L4[WIO] C3 F112 E112 S3 - L3[I] G1 - L2[W] R3 - L1[O] M3 - L0[I] G1 
[ 12] Utilization = 1.00 | pJ/Algorithmic-Compute =  601.916 | pJ/Compute =  601.916 | L4[WIO] M3 C3 F112 E112 S3 - L3[I] G1 - L2[W] G1 - L1[O] R3 - L0[I] G1 
[ 14] Utilization = 1.00 | pJ/Algorithmic-Compute =  427.387 | pJ/Compute =  427.387 | L4[WIO] C3 F112 E112 S3 - L3[I] R3 - L2[W] G1 - L1[O] M3 - L0[I] G1 
[ 15] Utilization = 1.00 | pJ/Algorithmic-Compute =  427.254 | p



Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Algorithmic-Compute =  254.935 | pJ/Compute =  254.935
TIMELOOP-MAPPER COMMAND:  timeloop-mapper /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/arch-dense.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/workloads/conv1/conv1-1.0-1.0-workload.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/mapper.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/dense/ART-dense.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/dense/ERT-dense.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/sparsity-dense.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/constraints/conv1-constraints.yaml
Completed dense for layer conv1 with iact sparsity = 1.0 and weight sparsity = 1.0
Modelling gating for

 12] Utilization = 1.00 | pJ/Algorithmic-Compute =  374.098 | pJ/Compute =  374.098 | L4[WIO] M3 C3 F112 S3 E112 - L3[I] G1 - L2[W] G1 - L1[O] R3 - L0[I] G1 
[  2] Utilization = 1.00 | pJ/Algorithmic-Compute =  401.583 | pJ/Compute =  401.583 | L4[WIO] F112 S3 E112 - L3[I] M3 R3 - L2[W] G1 - L1[O] C3 - L0[I] G1 
[ 12] Utilization = 1.00 | pJ/Algorithmic-Compute =  371.806 | pJ/Compute =  371.806 | L4[WIO] M3 C3 S3 F112 E112 - L3[I] G1 - L2[W] G1 - L1[O] R3 - L0[I] G1 
[  4] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the last valid mapping, terminating search.
[  0] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the last valid mapping, terminating search.
[  5] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the last valid mapping, terminating search.
[  9] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the last valid mapping, terminating search.
[  8] STATEMENT: 300 invalid mappings (0 fanout, 30



Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Algorithmic-Compute =  254.935 | pJ/Compute =  254.935
TIMELOOP-MAPPER COMMAND:  timeloop-mapper /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/arch-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/workloads/conv1/conv1-1.0-1.0-workload.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/mapper.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/gating/ART-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/gating/ERT-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/sparsity-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/constraints/conv1-constraints.yaml
Completed gating for layer conv1 with iact sparsity = 1.0 and weight sparsity = 1.0
Modelling ski

...



Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Algorithmic-Compute =  164.062 | pJ/Compute =  191.322
TIMELOOP-MAPPER COMMAND:  timeloop-mapper /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/arch-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/workloads/conv_dw_5/conv_dw_5-0.9641-0.88944-workload.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/mapper.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/gating/ART-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/gating/ERT-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/sparsity-gating.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/constraints/conv_dw_5-constraints.yaml
Completed gating for layer conv_dw_5 with iact sparsity = 0.9641 and weight sp

[  0] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the last valid mapping, terminating search.
[  5] Utilization = 1.00 | pJ/Algorithmic-Compute =  164.088 | pJ/Compute =  191.352 | L4[WIO] M128 C32 S3 F28 E28 - L3[I] G1 - L2[W] R3 - L1[O] C4 - L0[I] G1 
[ 10] Utilization = 1.00 | pJ/Algorithmic-Compute =  394.294 | pJ/Compute =  459.808 | L4[WIO] M128 C32 F28 S3 E28 - L3[I] R3 - L2[W] C2 - L1[O] C2 - L0[I] G1 
[  6] Utilization = 1.00 | pJ/Algorithmic-Compute =  394.294 | pJ/Compute =  459.808 | L4[WIO] M128 C32 F28 S3 E28 - L3[I] R3 - L2[W] G1 - L1[O] C4 - L0[I] G1 
[ 14] Utilization = 1.00 | pJ/Algorithmic-Compute =  394.294 | pJ/Compute =  459.808 | L4[WIO] M128 C32 F28 S3 E28 - L3[I] R3 - L2[W] C4 - L1[O] G1 - L0[I] G1 
[  3] Utilization = 1.00 | pJ/Algorithmic-Compute =  774.779 | pJ/Compute =  903.512 | L4[WIO] C128 F28 E28 S3 R3 - L3[I] M16 - L2[W] G1 - L1[O] M8 - L0[I] G1 
[  9] STATEMENT: 300 invalid mappings (0 fanout, 300 capacity) found since the la



Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Algorithmic-Compute =  164.088 | pJ/Compute =  191.352
TIMELOOP-MAPPER COMMAND:  timeloop-mapper /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/arch-skipping.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/workloads/conv_dw_5/conv_dw_5-0.9641-0.88944-workload.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/mapper.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/skipping/ART-skipping.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/ert-art/skipping/ERT-skipping.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/design/sparsity-skipping.yaml /home/workspace/2022.micro.artifact/notebook/output-sparsity-experiments/mapper/constraints/conv_dw_5-constraints.yaml
Completed skipping for layer conv_dw_5 with iact sparsity = 0.9641

[  6] Utilization = 1.00 | pJ/Algorithmic-Compute =  709.840 | pJ/Compute =  709.840 | L4[WIO] C32 F28 E28 - L3[I] C4 M2 - L2[W] M16 - L1[O] M4 - L0[I] G1 
[ 10] Utilization = 1.00 | pJ/Algorithmic-Compute =  231.922 | pJ/Compute =  231.922 | L4[WIO] C8 F28 E28 - L3[I] M128 C4 - L2[W] C4 - L1[O] G1 - L0[I] G1 
[  1] Utilization = 1.00 | pJ/Algorithmic-Compute =  231.759 | pJ/Compute =  231.759 | L4[WIO] C8 F28 E28 - L3[I] M128 - L2[W] C4 - L1[O] C4 - L0[I] G1 
[ 14] Utilization = 1.00 | pJ/Algorithmic-Compute =  710.303 | pJ/Compute =  710.303 | L4[WIO] C32 F28 E28 M32 - L3[I] C4 - L2[W] M2 - L1[O] M2 - L0[I] G1 
[ 12] Utilization = 1.00 | pJ/Algorithmic-Compute =  709.840 | pJ/Compute =  709.840 | L4[WIO] C128 F28 E28 M2 - L3[I] M16 - L2[W] M2 - L1[O] M2 - L0[I] G1 
[  9] Utilization = 1.00 | pJ/Algorithmic-Compute =  262.374 | pJ/Compute =  262.374 | L4[WIO] C16 F28 E28 M16 - L3[I] M4 - L2[W] C8 - L1[O] M2 - L0[I] G1 
[ 13] Utilization = 1.00 | pJ/Algorithmic-Compute =  710.781 | pJ/

[ 15] Utilization = 1.00 | pJ/Algorithmic-Compute =  291.988 | pJ/Compute =  291.988 | L4[WIO] M16 C16 F28 E28 - L3[I] M4 C8 - L2[W] G1 - L1[O] M2 - L0[I] G1 
[  5] Utilization = 1.00 | pJ/Algorithmic-Compute =  259.919 | pJ/Compute =  259.919 | L4[WIO] M16 C8 F28 E28 - L3[I] M4 C16 - L2[W] G1 - L1[O] M2 - L0[I] G1 
[ 15] Utilization = 1.00 | pJ/Algorithmic-Compute =  261.814 | pJ/Compute =  261.814 | L4[WIO] C16 F28 E28 M16 - L3[I] M4 C8 - L2[W] G1 - L1[O] M2 - L0[I] G1 
[  5] Utilization = 1.00 | pJ/Algorithmic-Compute =  229.745 | pJ/Compute =  229.745 | L4[WIO] C8 F28 E28 M16 - L3[I] M4 C16 - L2[W] G1 - L1[O] M2 - L0[I] G1 
[ 15] Utilization = 1.00 | pJ/Algorithmic-Compute =  231.759 | pJ/Compute =  231.759 | L4[WIO] C8 F28 E28 - L3[I] M128 - L2[W] G1 - L1[O] C16 - L0[I] G1 
[ 14] Utilization = 1.00 | pJ/Algorithmic-Compute =  261.071 | pJ/Compute =  261.071 | L4[WIO] C16 F28 E28 - L3[I] M8 C4 - L2[W] C2 - L1[O] M16 - L0[I] G1 
[ 10] Utilization = 1.00 | pJ/Algorithmic-Compute =  2